<p></p><font size=8 face="courier" color="magenta">Embed</font>

This notebook creates the widget inside of the "Visualize" section of the [Cytoscape app](../Cytoscape.ipynb).

# <font color="gray">Setup Notebook</font>

## Load Stuff

`%cd` sets the path to that of Cytoscape.ipynb

In [1]:
""" DEBUG 
%cd -q  "/root/Documents/Apps"
%run "Setup_Notebook/default.ipynb"
%cd -q -
"""

<IPython.core.display.Javascript object>

## Create <font color="blue">Python</font> <font color="magenta">Graph</font> object

Here we create the initial <font color="magenta">Graph</font> class. This class is used to creat and manage the networks visualization. This section only consists of the initialization <font color="magenta">Graph</font> and some of it's basic functions. Most of <font color="magenta">Graph</font> functionality is actually implemented inside of the **Add Widgets** section

In [ ]:
class Graph():
        pass

In [ ]:
Graph.html = type('html',(),{})

### <font color="darkKhaki">Create HTML for the CY element</font>  <small>(holds the graph)</small>

In [ ]:
Graph.html.cy=HTML("""
<style>.cy{ 
    width: 100%; 
    height: 40vw;
    border-style: solid;
    border-width: 1px;
}</style>
<div id='cy' class='cy'></div>
""")

### <font color="darkKhaki">Create HTML for Table element</font>

In [ ]:
Graph.html.table=HTML("""
<table id="table" class="table table-striped table-bordered">
    <thead><tr id="table_head"></tr></thead>
    <tbody id="table_body"></tbody>    
</table>
""")

# <font color="orange">Javascript Functions</font>

## <font color="orange">Start Javascript</font>

`Graph.js` holds all of the js code. All of the js code (that not written inside a function with `def`) must be ran at once or else errors will occure.

We building a `Graph.js.text` which hold all the javascript as text, which is run by `Graph.js.load()`. However, during development and debugging, it's easier to run the js code a a few cells at a time.

In [ ]:
Graph.js = type("javascript",(),{})()

Initialize the javascript object cy, which handles all the manipulations of the html canvas (also called 'cy')

In [ ]:
Graph.js.text = """
var cy;
require(['cytoscape'], function(cytoscape){ 
    cy = cytoscape({ container: document.getElementById('cy') });
"""    
Graph.js.end = "});"

<font color="orange">Graph.js.load</font>

In [ ]:
def loadJS():
    JS( Graph.js.text + Graph.js.select  +Graph.js.end )
Graph.js.load = loadJS
#Graph.js.load() ### DEBUG ###

initial settings for 'cy' 

In [ ]:
### For Changing highlight features:
#def selectColor(self):

#checkit = cy.$(':selected')
#checkit.style("overlay-color","gray")
#checkit.style("overlay-opacity",".3")
#checkit.style("overlay-padding","8px")

In [ ]:
js = """
cy.io = {}       // <-- where we store all the parameters/variables we create        
cy.maxZoom(14)   // <-- Determine how much window zooms in and out
cy.minZoom(.4)

// Create default style for nodes and edges
cy.io.style = {}
cy.io.style.node = { 'overlay-color': 'gray', 'overlay-padding': '2px', 'background-color': 'gray', 'text-background-color':'white',
                        'text-border-width':'2px', 'text-background-padding':'4px', 'z-index-compare':'manual' }
cy.io.style.edge = { 'overlay-color': 'gray', 'overlay-padding': '8px', 'line-color': 'gray', 'curve-style': 'bezier', 'text-background-color':'white',
                        'text-border-width':'2px', 'text-background-padding':'4px', 'z-index-compare':'manual' }
"""
Graph.js.text = Graph.js.text + js
#JS( js ) ### DEBUG ###

Make node/edge highlighted when selected.  We save this functionality as <font color="blue">cyOverlay</font> because it's later used when saving a graph to the <font color="DarkKhaki">CY.HTML </font> format.

In [ ]:
js = """
cy.on("select", 'node, edge' , function(){     //<-- highlight node when selected
    this.style( 'overlay-opacity', .4 );
});
cy.on("unselect", 'node, edge' , function(){   //<-- unhighlight when not selected
    this.style( 'overlay-opacity', 0 );
});
"""
Graph.cyOverlay = js

In [ ]:
Graph.js.text = Graph.js.text + js
#JS( js ) ### DEBUG ###

<font color="orange">Graph.js.select</font> holds all of the selections event code

When selecting/unselecting elements, Save selection search (for computational efficiency) 

In [ ]:
Graph.js.select = """
cy.on('select unselect', 'node, edge', function(event){ 
cy.io.selection= cy.$(':selected')  
});"""

Helper function used to add things to <font color="orange">Graph.js.select</font>

In [ ]:
def selectAdd(text):
    index= Graph.js.select.rfind("\n")
    Graph.js.select = Graph.js.select[:index] + text + Graph.js.select[index:] 
Graph.js.selectAdd = selectAdd

## <font color="orange">Set Styles</font>

<font color="orange">cy.io.style.set</font> sets the style of the node/edge when it's added to the graph. If the parameter, <font color="blue">self</font> is true (default false) then function works the other way: 
>  <font color="orange">cy.io.style</font> $\leftarrow$ element.style() 

In [ ]:
js="""
cy.io.style.set = function(ele, self=false){
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;
    
    for( var property in ele.style() ){ 
        if(  !/[A-Z]/.test(property) && !property.includes("pie") && ele.style(property) != "NaNpx" && !property.includes("overlay") &&
                                        style[property] != ele.style(property)  ){   //<--  remove cytoscape.js errors and repitition
           if( self == false ){                  //<-- set properties of element
                ele.style( property, style[property] )
            }else{                               //<-- set properties of style *using* element
                style[ property ] = ele.style( property ) 
            }    
       }
    }
};
"""
Graph.js.text = Graph.js.text + js
#JS( js ) ### DEBUG ###

## <font color="orange">addNode</font>

<font color="orange">cy.io.addNode</font>  adds new nodes to the network when the clicks and empty part of the graph.

In [ ]:
js="""
cy.io.addNode = function(event){
  if( event.target === cy ){                                                          // <-- If we tapped on the background
      var ele = cy.add([ {  group: "nodes", position: event.position  } ]);          //<-- add node    
      cy.io.style.set( ele )
  }; 
};
cy.on('tap', cy.io.addNode );
"""
Graph.js.text = Graph.js.text + js
#JS( js ) ### DEBUG ###

## <font color="orange">addEdge</font>

<font color="orange">cy.io.addEdge</font> adds a new edge when the use double clicks one nodes and then double clicks another node. <font color="orange">addEdge</font> check for the double clicking and adds a new edge if necessary.

In [ ]:
js="""
cy.io.click_hist = ["1", "2", "3", "4"];
cy.io.click_time = [8, 6, 4, 2];

cy.io.addEdge = function(event){                                    // <-- on "tapping" canvas
    cy.io.click_hist.shift();     //<-- Remove oldest click
    
    //Update Click times
    var d = new Date()
    cy.io.click_time.shift()
    cy.io.click_time.push( d.getTime()/1000 )
    
    if( event.target !== cy && event.target.isNode() ){               // <-- If we tapped a Node
      cy.io.click_hist.push( event.target.id() );                   // <-- Add node id to click history

      if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] && \
          cy.io.click_time[1] - cy.io.click_time[0] < 1 && cy.io.click_time[3] - cy.io.click_time[2] < 1  ){ //<-- If 2 double clicks
          
          var ele= cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]);    //<-- add edge
          cy.io.style.set( ele )                                                                            //<-- add style of new edge
      };

    }else{                                                 //<-- if we didn't click a node
      cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
    }; 
};
cy.on( 'tap' , cy.io.addEdge );
"""
Graph.js.text = Graph.js.text + js
#JS("cy.removeListener( 'tap', cy.io.addEdge) ") ### DEBUG ##
#JS( js ) ### DEBUG ###

## <font color="orange">Change Style for Element</font>

<font color="orange">cy.io.style.change</font> is used to edit the style elements.

This process is a little involved because the style variables <font color="orange">cy.io.style.node</font> and <font color="orange">cy.io.style.edge</font> must also be set to the style of the current element, and then changed.

In [ ]:
js="""
cy.io.style.change = function(ele, property, value ){
    cy.io.style.set( ele, self=true )                                     // Make style same as node type
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;   // get style type
    
    typeof(property)
    if( typeof(property)=="string" ){                                     // handle changing a single style property
        style[ property ] = value                                             // change style value
        ele.style( property, value )                                          // change element's value
    }else{                                                                // Multiple properties
        for(var i=0; i<property.length; i++){
            style[ property[i] ] = value[i]
            ele.style( property[i], value[i] )  
        }
    }
};
"""
Graph.js.text = Graph.js.text + js
#JS( js ) ### DEBUG ###

## <font color="orange">Get Default Node/Edge Style</font>

the variables <font color="orange">cy.io.default.node/edge</font> hold the default stylesheets create when the networks starts.

After setting up everything else (above) we create some dummy data, set <font color="orange">cy.io.style</font> to their style, and then set <font color="orange">cy.io.default</font> to that.

In [ ]:
js="""
var eles = cy.add([  {data: { id: 'a' }},  {data: { id: 'b' }},  {data: { id: 'ab', source: 'a', target: 'b'}}  ])
cy.io.style.set( eles[0] ) 
cy.io.style.set( eles[2] )
cy.io.style.set(eles[0], self=true )
cy.io.style.set(eles[2], self=true )
cy.io.default = {}
cy.io.default.node = jQuery.extend(true, {}, cy.io.style.node );
cy.io.default.edge = jQuery.extend(true, {}, cy.io.style.edge );
cy.remove(eles)
"""
Graph.js.text = Graph.js.text + js
#JS( js ) ### DEBUG ###

# <font color="Navy">Editor</font> <font color="green">Widgets</font> 

The purpose of the <font color="green">widgets</font>  section is to create all the basic "tools" that are used to manipulate the cytoscape canvas. We create the tools and the functions that allow them to do their job. Later, in the **"Display Boxes"** section, we combine to the tools together to build a pretty layout

In [ ]:
Graph.w = type("widget_list",(),{})()
Graph.w.time = 0 #<-- used to measure time between update events (see size for example)
Graph.w.vars = {}#<-- used to hold variable classes that a user creates

## <font color="DarkKhaki">Node</font> and <font color="purple">Edge</font> <font color="green">Widgets</font> 

### <font color="green">Color </font> 

In [ ]:
Graph.w.color = ipw.ColorPicker(  )
#Graph.w.color

In [ ]:
def color(b):
    values = "['"+Graph.w.color.value+"','"+Graph.w.color.value+"']"
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, ["background-color","line-color"], """+values+"""  )
    }
    """))          
Graph.w.color.observe( color, names='value' )

### <font color="green">Size</font> 

In [ ]:
Graph.w.size = ipw.IntSlider(value=30, min=1, max=200, continuous_update=True )
#Graph.w.size 

In [ ]:
def size(b):
    values = "['"+str(Graph.w.size.value)+"','"+str(Graph.w.size.value)+"']" #<-- values for changing node's style
    display(Javascript("""
    for( var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isNode() ){
            cy.io.style.change( ele, ["width","height"], """+values+"""   )   
        }else{
            cy.io.style.change( ele, "width", '"""+str(Graph.w.size.value/6)+"""'   )   
        }
    }
    """))           
Graph.w.size.observe( size, names='value' )

### <font color="green">Opacity</font>  

In [ ]:
Graph.w.opacity = ipw.FloatSlider(value=.5, min=0, max=1, step=.05, continuous_update=True )
#Graph.w.opacity 

In [ ]:
def opacity(b):
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[ i ]
        cy.io.style.change( ele, "opacity", '"""+str(Graph.w.opacity.value)+"""'  )
    }
    """))           
Graph.w.opacity.observe( opacity, names='value' )

### <font color="green">Z-Index Labels Behind/In-Front</font> <font color="red">Glitchy</font>

In [ ]:
Graph.w.z_index = ipw.IntSlider(value=0, min=0, max=5, continuous_update=True )
#Graph.w.z_index

In [ ]:
def z_index(b):
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[ i ]
        cy.io.style.change( ele, "z-index", '"""+str(Graph.w.z_index.value)+"""'  )
    }
    """))               
Graph.w.z_index.observe( z_index, names='value' )

## <font color="green">General Widgets</font> 

### <font color="green">Undo Delete </font> 

In [ ]:
Graph.w.undo = ipw.Button(description='Undo',button_style='primary', disabled= True )
#Graph.w.undo

In [ ]:
def undo(b):
    display(Javascript("cy.add( cy.io.last_deleted );"))   #<-- Add last deleted element
    b.disabled = True
Graph.w.undo.on_click( undo )

### <font color="green">Delete</font> 

In [ ]:
Graph.w.delete = ipw.Button(description='Delete',button_style='danger')
#Graph.w.delete

In [ ]:
def delete(b):
    display(Javascript("cy.io.last_deleted = cy.io.selection.remove();"))   #<-- delete elements
    Graph.w.undo.disabled = False
Graph.w.delete.on_click( delete )

### <font color="green">Background Color </font> 

In [ ]:
Graph.w.background = ipw.Button( button_style='danger', icon='image', layout= ipw.Layout(width='auto') )
#Graph.w.background

In [ ]:
def background(b):
    display(Javascript(f"document.getElementById('cy').style.backgroundColor = '{Graph.w.color.value}'"))  
Graph.w.background.on_click( background )

### <font color="green">Search</font> 

In [ ]:
Graph.w.search = widgets.Text( description='Search :', continuous_update=False )
#Graph.w.search

In [ ]:
def search(b):
    display(Javascript(f"""
    cy.io.selection.unselect()
    cy.$('{Graph.w.search.value}').select()
    """))   
Graph.w.search.observe( search, names='value' )

### <font color="green">Search Select</font>

In [ ]:
Graph.w.searchSelect = ipw.ToggleButton( icon='eye', layout= ipw.Layout(width='auto')  )
#Graph.w.searchSelect

In [ ]:
def searchSelect(b):
    if Graph.w.searchSelect.value:   #<-- when activated
        Graph.w.searchSelect.icon = 'eye-slash'
        display(Javascript("""
        cy.io.to_hide = cy.elements().difference( cy.io.selection )
        cy.io.to_hide.style("visibility", "hidden")
        """))
    else: #<-- when not clicked
        Graph.w.searchSelect.icon = 'eye'
        display(Javascript("""cy.io.to_hide.style("visibility", "visible")"""))
Graph.w.searchSelect.observe( searchSelect, names='value' )

##  <font color="salmon">Label</font> <font color="green">Widgets</font> 

### <font color="salmon">Label Text</font>

In [ ]:
Graph.w.label = widgets.Text( continuous_update=True )
#Graph.w.label

In [ ]:
def label(b):
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, "label", '"""+Graph.w.label.value+"""'  )
    }
    """))           
Graph.w.label.observe( label, names='value' )

### <font color="salmon">Label Color</font>

In [ ]:
Graph.w.label_color = ipw.ColorPicker( )
#Graph.w.label_color

In [ ]:
def labelColor(b):
    js_var = "['"+Graph.w.label_color.value+"','"+Graph.w.label_color.value+"']"
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, ['color','text-border-color'], """+js_var+""" )
    }
    """))          
Graph.w.label_color.observe( labelColor, names='value' )

### <font color="salmon">Label Size</font>

In [ ]:
Graph.w.label_size = ipw.IntSlider(value=16, min=6, max=60)
#Graph.w.label_size

In [ ]:
def labelSize(b):
    display(Javascript("""
    for( var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, 'font-size', '"""+str(Graph.w.label_size.value)+"""px' )
    }
    """))          
Graph.w.label_size.observe( labelSize, names='value' )

### <font color="salmon">Box</font>  

In [ ]:
Graph.w.label_box = ipw.ToggleButton( button_style='primary', description="Box", layout= ipw.Layout(width='auto') )
#Graph.w.label_box

In [ ]:
def labelBox(b):
    js_var = "['1','1']" if Graph.w.label_box.value else "['0','0']"
    display(Javascript("""
    for( var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, ['text-background-opacity','text-border-opacity'], """+js_var+""" )
    }
    """))
Graph.w.label_box.observe( labelBox, names='value' )

### <font color="DarkKhaki">Node</font> <font color="salmon">Label Position</font>  

In [ ]:
Graph.w.label_pos = ipw.Dropdown( options=['center','top','bottom','left','right'] )
#Graph.w.label_pos

In [ ]:
def labelPos(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        var label_pos = '"""+Graph.w.label_pos.value+"""'
        if( ele.isNode() ){
            if( label_pos == 'center'  ){
                cy.io.style.change( ele, ['text-halign','text-valign'], ['center','center'] )
            }else if( label_pos=='top' || label_pos=='bottom'){
                cy.io.style.change( ele, ['text-halign','text-valign'], ['center', label_pos] )                    
            }else{
                cy.io.style.change( ele, ['text-halign','text-valign'], [ label_pos, 'center'] )                
            }
        } 
    }
    """))           
Graph.w.label_pos.observe( labelPos, names='value' )

## <font color="DarkKhaki">Node Shape</font> 

In [ ]:
Graph.w.shape = ipw.Dropdown( options=['ellipse','triangle','rectangle','rhomboid','diamond','pentagon','hexagon',
                                       'concavehexagon','heptagon','octagon','star','tag','vee'], continuous_update=True )
#Graph.w.shape

In [ ]:
def shape(b):
    display(Javascript("""
    for( var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isNode() ){
            ele.style( "shape", '"""+Graph.w.shape.value+"""' )
        } 
    }
    """))           
Graph.w.shape.observe( shape, names='value' )

## <font color="purple">Edge</font> <font color="green">Widgets</font> 

### <font color="purple">Edge Arrow Start</font> 

In [ ]:
Graph.w.edgeStart = ipw.Dropdown( options=['none','triangle','tee','diamond','square','circle','vee','triangle-tee'] )
#Graph.w.start

In [ ]:
def edgeStart(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isEdge() ){
            ele.style( "source-arrow-color", ele.style("line-color")  )
            ele.style( "source-arrow-shape", '"""+Graph.w.start.value+"""' )
        } 
    }
    """))           
Graph.w.edgeStart.observe( edgeStart, names='value' )

### <font color="purple">Edge Arrow End</font> 

In [ ]:
Graph.w.edgeEnd = ipw.Dropdown( options=['none','triangle','tee','diamond','square','circle','vee','triangle-tee'] )
#Graph.w.end

In [ ]:
def edgeEnd(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isEdge() ){
            ele.style( "target-arrow-color", ele.style("line-color")  )
            ele.style( "target-arrow-shape", '"""+Graph.w.end.value+"""' )
        } 
    }
    """))           
Graph.w.edgeEnd.observe( edgeEnd, names='value' )

### <font color="purple">Edge Line Style</font> 

In [ ]:
Graph.w.lineStyle = ipw.Dropdown( options=['solid','dotted','dashed'] )
#Graph.w.lineStyle

In [ ]:
def lineStyle(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isEdge() ){
            ele.style( "lineStyle", '"""+Graph.w.lineStyle.value+"""' )
        } 
    }
    """))           
Graph.w.lineStyle.observe( lineStyle, names='value' )

## <font color="hotpink">Variables</font>  <font color="green">Widgets</font> 

**<font color="red">IDEA :</font>** To "hide" widgets just change width's and heights, after changing visibility

In [ ]:
Graph.w.vars = {}

In [ ]:
Graph.w.vars 

### <font color="hotpink">Change ID</font>  <font color="red">Not Implemented</font>

In [ ]:
Graph.w.id = widgets.Text( description='ID :', continous_update = True  )#, continous_update = False )
#Graph.w.id

In [ ]:
def setId(b):
    pass
Graph.w.id.observe( setId )

### <font color="hotpink">Create New  Variable</font>  

In [ ]:
Graph.w.var_create_type= ipw.Select( options=['Number', 'Text'], value='Number', layout=ipw.Layout(width='10%',height='40px',min_width='50px') )
Graph.w.var_create_text= ipw.Text(placeholder='Variable Name:')
Graph.w.var_create_btn = ipw.Button( button_style='primary', description="Create", layout= ipw.Layout(width='auto') )

Graph.w.var_create = ipw.HBox([ Graph.w.var_create_type, Graph.w.var_create_text, Graph.w.var_create_btn ])
Graph.w.var_create.layout.align_items = 'center'
#Graph.w.var_create

In [ ]:
def createVar(b):
    if Graph.w.var_create_text.value not in Graph.w.vars.keys() and  Graph.w.var_create_text.value!='' :
        Graph.w.vars[ Graph.w.var_create_text.value ] = Graph.w.var_create_type.value
    Graph.w.var_add_name.reset()
Graph.w.var_create_btn.on_click( createVar )

### <font color="hotpink">Show Variables</font> 

In [ ]:
Graph.w.varBox = ipw.VBox([])
#Graph.w.varBox

In [ ]:
# print(Graph.w.vars)

We Create a way to add variables to the HBox

In [ ]:
def add( label, value=None ):   
    
    if label not in Graph.w.vars : #<-- don't add stuff that's not part of the dataset
        return
    
    # We create the row (HBox) that is added to the box of variables
    var_label = ipw.HTML("<font size=3 style='justify-content: center;'>"+label+" : </font>") 
    var_value = ipw.Text(continuous_update=True) if Graph.w.vars[label]=='Text' else ipw.FloatText(continuous_update=True)
    if value != None :
        var_value.value = value 
    var_btn   = ipw.Button( button_style='danger', description="Remove", layout= ipw.Layout(width='auto') )
    var = ipw.HBox([ var_label, var_value, var_btn ])
            
    # now we setup remove button
    var_btn.label = label
    def remove(b):
        # Remove variable object from  varBox
        child_idx = [ x.children[1].label for x in list(Graph.w.varBox.children) ].index(b.label)  #<- index of child
        childs = list(Graph.w.varBox.children)
        childs.pop( child_idx )
        Graph.w.varBox.children = tuple(childs)
        Graph.w.var_add_name.reset()  #<-- reset options
        # remove data object from selected elments
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.removeData('"""+b.label+"""')
        }
        """))  
    var_btn.on_click( remove )
    
    # add a Variable Updating Function
    var_value.label = label
    def update(change):
        val =  "'"+change['new']+"'" if type(change['new'])== type('') else str(change['new'])
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.data('"""+change['owner'].label+"""', """+val+""" )
        }
        """))
    var_value.observe( update, names='value')        
        
    # Add the row to the varBox
    Graph.w.varBox.children = Graph.w.varBox.children + (var,)
        
Graph.w.varBox.add = add  

### <font color="orange">Retreive Variables</font> 

<font color="red">This function could be made a lot shorter</font>

In [ ]:
#JS("cy.removeListener('select unselect', 'node, edge', cy.io.getVars )")  ### DEBUG  ###
js="""
var selection = cy.io.selection
py.execute("Graph.w.varBox.children = ()")

if( selection.length == 0 ){ py.execute('Graph.w.var_add_name.reset(True)'); return;} 

cy.io.vars = []
cy.io.varTypes = []

if( selection.length == 1 ){        
    var data = selection[0].data()
    var data_vars = Object.keys(data).slice(1,)
    for( i in data){            
        if( data[i] != undefined && i!='id' ){
            cy.io.vars.push(i)
            var val_js = ( typeof(data[i])=="number" ) ? String(data[i]) : '"'+data[i]+'"'
            py.execute('Graph.w.varBox.add("'+i+'",'+val_js+')')

            var varType =  ( typeof(data[i])=="number" ) ? "number" : "text"
            cy.io.varTypes.push( varType )
        }
    }
}else{

    for( var i=0; i < selection.length; i++ ){
        var ele = selection[i]
        var data = ele.data()
        for( key in data ){
            if( key!='id' && !cy.io.vars.includes(key) && data[key]!=undefined ){
                cy.io.vars.push( key )

                var varType =  ( typeof(data[key])=="number" ) ? "number" : "text"
                cy.io.varTypes.push( varType )
            }
        }
    }
    for( show in cy.io.vars ){
        py.execute('Graph.w.varBox.add("'+cy.io.vars[show]+'")')
    }
}
py.execute('Graph.w.var_add_name.reset()')
"""
Graph.js.selectAdd( js )
#JS( "cy.on('select unselect', 'node, edge', function(event){"+js+"};") ### DEBUG ###

### <font color="hotpink">Add Variable</font> 

In [ ]:
Graph.w.var_add_name = ipw.Dropdown( options=list(Graph.w.vars.keys()), value=None) #,layout= ipw.Layout(width='10%') )
#Graph.w.var_add_name

In [ ]:
def wReset(disable=False):   #<-- disable is used to disable the add button when nothing's selected
    boxVars = [ x.children[1].label for x in list(Graph.w.varBox.children) ]  #<-- gets list of vriables that are currently shown
    Graph.w.var_add_name.options =  list(set(Graph.w.vars.keys()) - set(boxVars))   #<-- removes list of shown variables from those that are addible    
    Graph.w.var_add_btn.disabled = True if( len(Graph.w.var_add_name.options)==0 or disable) else False
Graph.w.var_add_name.reset = wReset

In [ ]:
Graph.w.var_add_btn = ipw.Button(description="Add", button_style='success', layout= ipw.Layout(width='auto') )
#Graph.w.var_add_btn

In [ ]:
def addVar( b ):
    val = Graph.w.var_add_name.value
    Graph.w.varBox.add( val )
    val_js = "''" if Graph.w.vars[val]=='Text' else "0"
    Graph.w.var_add_name.reset()
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        ele.data( '"""+val+"""', """+val_js+""" )
    }
    """))
Graph.w.var_add_btn.on_click( addVar )

We add an obsercer to **varBox** which updates the options of the "add" dropdown depending on what variables are still not used

# <font color="Navy">Editor</font> <font color="orchid">Box</font>

Here we take the prviously created <font color="green">widgets</font> and we group them rogether into a pretty box.

In [ ]:
Graph.box = type("box",(),{})()

**Padding : ** \[ top/ right/ bottom/ left \]

## <font color="green">General</font> Box <small>(Outside of Sliders)</small>

In [ ]:
Graph.box.search = ipw.HBox([ Graph.w.search, Graph.w.searchSelect, ipw.HTML("<p>&nbsp&nbsp&nbsp&nbsp&nbsp</p>") ])
Graph.box.search.layout.justify_content = "center"
Graph.box.search.layout.padding = '0px 20px 5px 0px'
#Graph.box.search

In [ ]:
Graph.box.delete = ipw.HBox([ Graph.w.delete, Graph.w.undo ] )
Graph.box.delete.layout.justify_content = "center"
#Graph.box.delete

In [ ]:
Graph.box.general = ipw.VBox([Graph.box.search, 
                              Graph.box.delete ])
Graph.box.delete.layout.padding = '0px 0px 10px 0px'
#Graph.box.general

##  <font color="DarkKhaki">Node</font> <font color="purple">Edge</font> Box

We add some style/layout properties to the widgets we will use in the style box

In [ ]:
Graph.box.color = ipw.HBox([ ipw.Label('Color :'), Graph.w.color, 
                            ipw.Label('    '), Graph.w.background ])

In [ ]:
Graph.box.size = ipw.HBox([ipw.Label('Size :'), Graph.w.size ])
Graph.box.size.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.lineStyle = ipw.HBox([ipw.Label('Line Style :'), Graph.w.lineStyle ])
Graph.box.lineStyle.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.opacity = ipw.HBox([ipw.Label('See-Through :'), Graph.w.opacity ])
Graph.box.opacity.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.z_index = ipw.HBox([ipw.Label('Behind/In-Front :'), Graph.w.z_index ])
Graph.box.z_index.layout.padding = '10px 0px 0px 0px'

Node Properties

In [ ]:
Graph.box.shape = ipw.HBox([ipw.Label('Shape :'), Graph.w.shape ])
Graph.box.shape.layout.padding = '0px 0px 10px 0px'

Edge properites

In [ ]:
Graph.box.edge_start = ipw.HBox([ipw.Label('Start', layout=ipw.Layout(min_width='30px') ), Graph.w.edgeStart])

In [ ]:
Graph.box.edge_end = ipw.HBox([ ipw.Label('End',   layout=ipw.Layout(min_width='30px', margin='0px 0px 0px 5px') ), Graph.w.edgeEnd ])
Graph.box.edge_end.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.edge_style = ipw.HBox([ipw.Label('Style', layout=ipw.Layout(min_width='30px') ), Graph.w.lineStyle ])
Graph.box.edge_style.layout.padding = '10px 0px 0px 0px'

Now we create the style box

In [ ]:
Graph.box.style = ipw.VBox([Graph.box.color,
                            Graph.box.size,
                            Graph.box.opacity,
                            Graph.box.z_index,
                            ipw.HTML("<h4 style='color:DarkKhaki'>Node :</h4>"),
                            Graph.box.shape,
                            ipw.HTML("<h4 style='color:purple'>Edge :</h4>"),
                            Graph.box.edge_start,
                            Graph.box.edge_end,
                            Graph.box.edge_style,
                           ])
#Graph.box.style

## <font color="salmon">Label Box</font>

We add some style/layout properties to the widgets we will use in the label box

In [ ]:
Graph.box.label_text = ipw.HBox([ipw.Label('Text :'), Graph.w.label ])

In [ ]:
Graph.box.label_color = ipw.HBox([ipw.Label('Color :'), Graph.w.label_color])
#Graph.box.label_color.layout.justify_content = "space-between"
#Graph.box.label_color

In [ ]:
Graph.box.label_size = ipw.HBox([ipw.Label('Size :'), Graph.w.label_size  ])
Graph.box.label_size.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.label_pos = ipw.HBox([ipw.Label('Position :'), Graph.w.label_pos])
Graph.box.label_pos.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.label_box = ipw.HBox([Graph.w.label_box  ])
Graph.box.label_box.layout.padding = '10px 0px 0px 0px'
Graph.box.label_box.layout.justify_content = 'center'
#Graph.box.label_box

Create the label box

In [ ]:
Graph.box.label = ipw.VBox([Graph.box.label_text,
                            ipw.HTML("<hr style='padding: 2px; margin: 2px'>"),
                            Graph.box.label_color,
                            Graph.box.label_size,
                            Graph.box.label_box
                           ])
#Graph.box.label

## <font color="hotPink">Variables Box</font>

In [ ]:
Graph.box.var_add = ipw.HBox([ Graph.w.var_add_name, Graph.w.var_add_btn])
Graph.box.var_add.layout.justify_content = 'center'

In [ ]:
Graph.box.vars = ipw.VBox([Graph.w.var_create,
                           ipw.HTML("<hr style='padding: 2px; margin: 2px'>"),
                           Graph.w.varBox,
                           ipw.HTML("<hr style='padding: 2px; margin: 2px'>"),
                           Graph.box.var_add
                           ])
#Graph.box.vars

## Combine Everything

In [ ]:
Graph.box.combo = ipw.Accordion(children=[ Graph.box.style ,Graph.box.label, Graph.box.vars])
Graph.box.combo.set_title(0, 'Style')
Graph.box.combo.set_title(1, 'Label')
Graph.box.combo.set_title(2, 'Variables')

Graph.box.editor = ipw.VBox([  Graph.box.general, Graph.box.combo ])
#Graph.box.editor

## <font color="orange">editSelector</font>

<font color="orange">editSelector</font> decides what widgets are displayed inside of the editor box *("My Edit Box")*. What's displayed depends on if there are 1/many of node/edges/both.

In [ ]:
js = """
    var check = 0;
    var check_var;
    cy.io.select_type = "nothing"


    cy.io.test = function(event){
        cy.$(':selected') = cy.$(':selected')                 //<-- Get selected elements
        cy.io.last_type = cy.io.select_type

        if( cy.$(':selected').length == 0  ){                 //<-- Do nothing if nothing's selected
            return
        }
        if( cy.$(':selected').length == 1 ){                  //<-- If selection lenght is 1
            //DEBUG py.execute("Graph.w.id.value = '"++"'")  //<- put selectors ID in box

            if( cy.$(':selected').isNode() && cy.io.last_type != "1 node" ){                      //<-- If it's a node
                cy.io.select_type = "1 node"
                //DEBUG py.execute("Graph.editor_box.widg = Graph.box.show('1 node')")
                IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box 
            }
            if( cy.$(':selected').isEdge() &&  cy.io.last_type != "1 edge" ){                     //<-- If it's an edge
                cy.io.select_type = "1 edge"
                //DEBUG py.execute("Graph.editor_box.widg = Graph.box.show('1 edge')")
                IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box
            }

        } else {                                           //<-- If Many elements
            console.log("many items")
        }

    };

    cy.on('select unselect', 'node, edge', cy.io.test );
"""
#Graph.js.selectAdd( js )
#JS( "cy.on('select unselect', 'node, edge', function(event){"+js+"};") ### DEBUG ###

# <font color="DeepSkyBlue">Table</font>

Create table head

In [ ]:
js ="""
var table_head = document.getElementById("table_head")
table_head.innerHTML = ''
for( var i in cy.io.vars ){
    table_head.innerHTML += '<th style="text-align: center"><h3>'+cy.io.vars[i]+'</h3></th>'
}
"""
Graph.js.selectAdd( js )
#JS( "cy.on('select unselect', 'node, edge', function(event){"+js+"};") ### DEBUG ###

put table data inside cells

In [ ]:
js ="""
    var table_body = document.getElementById("table_body")
    table_body.innerHTML = ""
    
    for( var i=0; i<cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        var data = ele.data()
        var row = document.createElement("tr");
        
        for( var j in cy.io.vars ){
            var cell = document.createElement("td");
            val = ( cy.io.varTypes[j]=="number" ) ? "Number(this.value)" : "this.value"
            addFunc = '"' + "cy.$id('" +data['id']+ "').data('" +cy.io.vars[j]+ "', " +val+ " )" +'"' 
            
            if( data[cy.io.vars[j] ]!=undefined ){
                cell.innerHTML = '<input type="'+cy.io.varTypes[j]+'" class="form-control" value="'+data[cy.io.vars[j]]+'" onkeyup='+addFunc+'  >' 
            }else{
                cell.innerHTML = '<input type="'+cy.io.varTypes[j]+'" class="form-control" onkeyup='+addFunc+'  >' 
            }
            row.appendChild(cell)
        }
        table_body.appendChild(row)
    }
"""
Graph.js.selectAdd( js )
#JS( "cy.on('select unselect', 'node, edge', function(event){"+js+"};") ### DEBUG ###

Create table style

In [ ]:
display(HTML("""<style>
#table_body tr td {
    padding: 2px;
}
#table_body tr td input{
    height: 100%;
    width: 100%;
    border: 0px;
    text-align: center;
}
</style>"""))

# <font color="limegreen">Load Data</font> <font color="green">Widgets</font> <font color="orange"> started</font>

## <font color="limeGreen">Load </font>  <font color="DarkKhaki">File</font> <font color="orange">Started</font>

In [ ]:
Graph.w.chooseFileType = ipw.Dropdown(options=["cy.html","sif"])
#Graph.w.chooseFileType

## <font color="limeGreen">Load </font>  <font color="DarkKhaki">CY.HTML </font><font color="red"> Not Implemented</font>

## <font color="limeGreen">Load </font><font color="blue">Data Frame</font><font color="red"> Not Implemented</font>

## <font color="limeGreen">Load </font><font color="Chocolate">NDEX</font> <font color="orange">started</font>

In [ ]:
# We create the Second widget for Downloading from a website
Graph.w.load_ndex_text = ipw.HTML("<h5>Paste the <a>NDEx URL</a> of the dataset to download :</h5>")
Graph.w.load_ndex_username = ipw.Text(description='Username :')
Graph.w.load_ndex_password = ipw.Password(description='Password :')
Graph.w.load_ndex_login = ipw.VBox([ Graph.w.load_ndex_username, Graph.w.load_ndex_password  ])
Graph.w.load_ndex_url = ipw.Text(description='URL :')

Graph.w.load_ndex_btn = ipw.Button( description="Load", button_style='primary'  )
Graph.box.load_ndex_btn_box = ipw.HBox([Graph.w.load_ndex_btn  ])
Graph.box.load_ndex_btn_box.layout.padding = '10px 0px 0px 0px'
Graph.box.load_ndex_btn_box.layout.justify_content = 'center'

Graph.w.load_ndex = ipw.VBox([Graph.w.load_ndex_text, ipw.HBox([Graph.w.load_ndex_login, Graph.w.load_ndex_url]), Graph.box.load_ndex_btn_box ])
#Graph.w.load_ndex

In [ ]:
def load_ndex():
    pass
Graph.w.load_ndex_btn.on_click(load_ndex) 

## <font color="limeGreen">Load </font> Data File <font color="red">Not Implemented</font>

# <font color="crimson">Save Data</font> <font color="green">Widgets</font>  <font color="orange"> started</font>

In [ ]:
#renderJSON(json.loads(x))

## <font color="crimson">Save to </font>  <font color="DarkKhaki">CY.HTML </font><font color="red" size=2> small bug in comm function</font>

### <font color="green">Widget</font>

In [ ]:
Graph.w.htmlTitle = ipw.Text()
#Graph.w.htmlTitle

In [ ]:
Graph.w.htmlText = ipw.Textarea()
#Graph.w.htmlText

In [ ]:
Graph.w.saveHtml = ipw.Button( button_style="success", description="Save", layout= ipw.Layout(width='auto') )
#Graph.w.saveHtml

### <font color="orange">JS: Create Elements JSON</font>

In [ ]:
def saveHtml(b):
    display(Javascript("""
    //console.log("inside saveHtml")
    var toSend = {}
    toSend["elements"] = []; toSend["style"] = []; 
    
    // Add default styles:
    toSend["style"].push({ selector: 'node', style: cy.io.default.node })
    toSend["style"].push({ selector: 'edge', style: cy.io.default.edge }) 
    
    // Add Element data/style/position for all elements
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        toSend["elements"].push({ data: ele.data() , position: ele.position() })  //<-- Sends Node Info
        
        // Create node style
        var eleStyle = {}
        var template  =  ele.isNode() ? cy.io.default.node : cy.io.default.edge
        for( var property in ele.style() ){ 
            if( !/[A-Z]/.test(property) && !property.includes("pie") &&  ele.style(property) != "NaNpx" && !property.includes("overlay") &&
                                            template[property] != ele.style(property)  ){   //  remove errors, check that style different from default
                eleStyle[property] = ele.style( property ) 
            }
        }
        toSend["style"].push({ style: eleStyle , selector: '#'+ele.data('id') }) //<-- make sure to optimize this
    }
    
    cy.io.saveHtml.send( toSend )
    """))
Graph.w.saveHtml.on_click( saveHtml )

### <font color="blue">Comm Function : Finish JSON & Write file</font>

<font color="red">Have to fix ' marks</font>

In [ ]:
fromJS = "" ### DEBUG ###
def saveHtmlComm(comm, msg):
    @comm.on_msg
    def _recv(msg):
        global fromJS  ### DEBUG ###
        ! cp "/root/Documents/Apps/Cytoscape/Graph Template.html" "/root/Documents/Apps/Cytoscape/Graph.html"
        fromJS = msg['content']['data']

        with open("/root/Documents/Apps/Cytoscape/Graph.html","a") as f:
            f.write("<script> var cy = cytoscape({ container: document.getElementById('cy'), ")
            f.write( json.dumps(fromJS, indent=2)[1:] ) #<-- leave out first character, '{' 
            f.write(")"+ Graph.cyOverlay + Graph.cySelect ) #<-- add highlighting, and data visualization
            f.write("document.getElementById('title').innerHTML = '"+Graph.w.htmlTitle.value+"';\n")
            f.write("document.getElementById('description').innerHTML = '"+Graph.w.htmlText.value+"';\n </script>") #<-- Text and Title
            
get_ipython().kernel.comm_manager.register_target('save_html', saveHtmlComm)

In [ ]:
js ="""cy.io.saveHtml = Jupyter.notebook.kernel.comm_manager.new_comm('save_html')"""
Graph.js.text = Graph.js.text + js
JS( js ) ### DEBUG ###

### <font color="gray">Helper</font> <font color="orange">Display Data</font> in <font color="darkKhaki">CY.HTML</font>

<font color="orange">displayData</font> is a function used inside of the <font color="darkKhaki">CY.HTML</font> that we create. It displays a single node/edge's value to the user.

In [ ]:
Graph.cySelect="""
cy.on('select unselect', 'node, edge', function(event){ 
    var toFill = document.getElementById("data")
    toFill.innerHTML = "" //<-- resets values
    var toAdd = ""        //<-- where we write the html we will add

    if( cy.$(":selected").length == 1 ){ //<-- only run when 1 element's selected
        var data = cy.$(":selected")[0].data() 
        for( property in data ){
            if( property != "id"  &&  data[property] != undefined ){
                toAdd += "<h6 style='margin-bottom:0px'>"+property+" : </h6>"
                toAdd += "<p style='color:#2C3E50'><small>"+String(data[property])+"</small></p>"
            }
        }
    }
    toFill.innerHTML = toAdd
});
"""

### Scratch Paper

In [ ]:
"""display(Javascript(
for(var i=0; i < cy.io.selection.length; i++ ){
    var ele = cy.io.selection[i]
    cy.io.saveHtml.send( ele.data() )
}
))"""

In [ ]:
# ! cp "static/"

## <font color="crimson">Save to </font><font color="blue">Data Frame</font> <font color="red"> Not Implemented</font>

## <font color="crimson">Save to </font><font color="Chocolate">NDEX</font> <font color="red"> Not Implemented</font>

# <font color="limegreen">Load</font>/<font color="crimson">Save</font>  <font color="orchid">Box</font> <font color="orange"> Started</font>

## <font color="crimson">Save</font> <font color="DarkKhaki">CY.HTML</font>  <font color="orchid">Box</font> 

Setup <font color="green">widgets</font> in save box

In [ ]:
Graph.box.save_graph_title = ipw.HBox([ipw.HTML("<h5>Title  &nbsp</h5> "),Graph.w.htmlTitle])
Graph.box.save_graph_title.layout.align_items = "center"

In [ ]:
Graph.w.htmlText.layout.width = "99%"    #<-- make description box full width
Graph.w.htmlText.layout.height = "80px"  #<-- make description box correct height
Graph.w.saveHtml.layout.margin = "10px 0 0 0"

Description of what the <font color="DarkKhaki">CY.HTML</font> file is.

<font color="red">TO DO:</font> add a <font color="crimson">youtube</font> video about the file type

In [ ]:
Graph.box.save_graph_text = ipw.HTML("""<h5>You can save your graph as a <font color='darkKhaki'>cy.html</font> file.
<font color='darkKhaki'>cy.html</font> files have lots of cool features, as <a>explained in this video.</a> </h5>""")
Graph.box.save_graph_text.layout.padding = "0 15px 0 35px"

Make final box

In [ ]:
Graph.box.save_graph = ipw.VBox([Graph.box.save_graph_text,
                                 Graph.box.save_graph_title,
                                 ipw.HTML("<h5>Description</h5>"),
                                 Graph.w.htmlText,
                                 Graph.w.saveHtml
                                ])
Graph.box.save_graph.layout.align_items = "center"
#Graph.box.save_graph

##  <font color="limegreen">Load</font> <font color="orchid">Box</font> <font color="orange"> Started</font>

In [ ]:
Graph.w.load_options = ipw.ToggleButtons( options=['Python', 'File', 'NDEX'], button_style='success')
#Graph.w.load_options

In [ ]:
Graph.box.load = ipw.HBox([Graph.w.load_options])

## <font color="crimson">Save</font>  <font color="orchid">Box</font> <font color="orange"> Started</font>

Create save options widget

In [ ]:
Graph.w.save_options = ipw.ToggleButtons( options=['Python', 'File', 'NDEX'], button_style='danger')
Graph.w.save_options.layout.justify_content = "center"
Graph.w.save_options.layout.margin = "0 0 15px 0"
#Graph.w.save_options

In [ ]:
def saveOptions( b ):
    if Graph.w.save_options.value == "File" :
        Graph.box.save_graph.layout.display = None
    else:
        Graph.box.save_graph.layout.display = "none"
Graph.w.save_options.observe( saveOptions, names='value' )

Format widgets that go inside save box

In [ ]:
Graph.box.save_graph.layout.display = "none"

Create box

In [ ]:
Graph.box.save = ipw.VBox([Graph.w.save_options, Graph.box.save_graph])

##  <font color="orchid">Final Box</font>   <font color="orange"> Started</font>

In [ ]:
Graph.box.loadSave = ipw.Accordion(children=[Graph.box.load,Graph.box.save])
Graph.box.loadSave.set_title(0, 'Load')
Graph.box.loadSave.set_title(1, 'Save')
#Graph.box.loadSave

# Combine Everything

These are the cells that the user must have in their notebook (unless I figure out embedding)

We embed these cells into other notebooks using the **RunNewCell** Helper

## <font color="gray">(Mock Setup) <small>for debugging</small> </font>

In [ ]:
# My Canvas Box
#display( Graph.html.cy )

In [ ]:
# My Edit Box
#display( Graph.box.editor )

In [ ]:
#display( Graph.html.table )

In [ ]:
#display(Graph.box.loadSave)

## Run Everything

In [ ]:
RunNewCell("display( Graph.html.cy )")       #<-- displays GRAPH of Network
RunNewCell("display( Graph.box.editor ) ")   #<-- displays Editor Box
RunNewCell("display( Graph.html.table ) ")   #<-- displays Table
RunNewCell("display( Graph.box.loadSave) ")  #<-- displays Load Save 
RunNewCell("Graph.js.load()")